In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [5]:
from utils import load

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

AttributeError: module 'torch' has no attribute 'device'

In [7]:
classes = ["airplane","angel","basket","bear","belt","candle","crown","cat","bus","dog"]
root_location = "quickdraw/train_simplified/"
gen = load.ImageLoader(classes,root_location,read_size=32,batch_size=128)

AttributeError: module 'utils.load' has no attribute 'ImageLoader'

In [8]:
#Hyper parameters
num_epochs = 10
num_classes = gen.num_classes

NameError: name 'gen' is not defined

In [9]:
class LeNet(nn.Module):
    def __init__(self,num_classes):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.fc1   = nn.Linear(576, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, num_classes)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
model = LeNet(gen.num_classes).to(device).double()
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
accuracy = [0]
loss_stored = []
for epoch in range(num_epochs):            
    gen = load.ImageLoader(classes,root_location,read_size=32,batch_size=128)
    for i, (images, labels) in enumerate(gen):
        images = images.to(device)
        labels = labels
        
        # Forward pass
        outputs = model(images)
        
        loss = criterion(outputs.cpu(), labels)
        acc = 100 * np.mean(labels.data.numpy() == outputs.cpu().data.numpy ().T.argmax(axis =0))
        
        # Backward and optimize
        try:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        except:
            continue
        
        del images
        del labels
        del outputs
        torch.cuda.empty_cache()
        if (i+1) % 500 == 0:
            print ('Epoch [{}/{}], Step [{}], Loss: {:.4f},Accuracy:{:.4f}' .format(epoch+1, num_epochs, i+1, loss.item(),acc))
            #model.save_state_dict('mytraining.pt')
        if (i+1)%100==0:
            loss_stored.append(loss.item())
            accuracy.append((0.1*acc) + (0.9*accuracy[-1]))

Epoch [1/10], Step [500], Loss: 1.1425,Accuracy:64.8438
Epoch [1/10], Step [1000], Loss: 0.7761,Accuracy:77.3438
Epoch [1/10], Step [1500], Loss: 0.7679,Accuracy:76.5625
Epoch [1/10], Step [2000], Loss: 0.7446,Accuracy:75.0000
Epoch [1/10], Step [2500], Loss: 0.6557,Accuracy:82.8125
Epoch [1/10], Step [3000], Loss: 0.5879,Accuracy:82.8125
Epoch [1/10], Step [3500], Loss: 0.7097,Accuracy:76.5625
Epoch [1/10], Step [4000], Loss: 0.6435,Accuracy:79.6875
Epoch [1/10], Step [4500], Loss: 0.4167,Accuracy:85.9375
Epoch [1/10], Step [5000], Loss: 0.6115,Accuracy:82.8125
Epoch [1/10], Step [5500], Loss: 0.5406,Accuracy:80.4688
Epoch [1/10], Step [6000], Loss: 0.6176,Accuracy:76.5625
Epoch [1/10], Step [6500], Loss: 0.5172,Accuracy:81.2500
Epoch [1/10], Step [7000], Loss: 0.4511,Accuracy:85.1562
Epoch [1/10], Step [7500], Loss: 0.4987,Accuracy:79.6875
Epoch [1/10], Step [8000], Loss: 0.3768,Accuracy:85.1562
Epoch [1/10], Step [8500], Loss: 0.4095,Accuracy:87.5000
Epoch [1/10], Step [9000], Loss: